In [ ]:
import pandas as pd

In [ ]:
mimic_path = "/mnt/data/MIMIC/physionet.org/files/mimiciv/2.2/hosp/"
patients =  pd.read_csv(mimic_path + "patients.csv.gz", compression='gzip')
admissions = pd.read_csv(mimic_path + "admissions.csv.gz", compression='gzip')

In [ ]:
mimic_cxr_path = "/mnt/data/MIMIC-CXR-JPG/physionet.org/files/mimic-cxr-jpg/2.1.0/"
split = pd.read_csv(mimic_cxr_path + "mimic-cxr-2.0.0-split.csv.gz", compression='gzip')
metadata = pd.read_csv(mimic_cxr_path + "mimic-cxr-2.0.0-metadata.csv.gz", compression='gzip')

In [ ]:
mimic_cxr_df = pd.merge(split, metadata, on=['dicom_id', 'subject_id', 'study_id'])
len(mimic_cxr_df)

In [ ]:
mimic_df = pd.merge(patients, admissions, on='subject_id')
len(mimic_df)

In [ ]:
merged_df = pd.merge(mimic_cxr_df, mimic_df, on='subject_id', how='inner')
# merged_df.dropna(subset=['StudyDate', 'admittime', 'dischtime'], inplace=True)
print(len(merged_df))

merged_df['StudyDate'] = pd.to_datetime(merged_df['StudyDate'], format='%Y%m%d')
merged_df['admittime'] = pd.to_datetime(merged_df['admittime'])
merged_df['dischtime'] = pd.to_datetime(merged_df['dischtime'])

duplicate_rows = merged_df[merged_df.duplicated(subset=['subject_id', 'study_id', 'dicom_id'])]
filtered_duplicates = duplicate_rows[
    (duplicate_rows['StudyDate'] >= duplicate_rows['admittime'].dt.date) & 
    (duplicate_rows['StudyDate'] <= duplicate_rows['dischtime'].dt.date)
]

filtered_duplicates = filtered_duplicates.drop_duplicates(subset=['subject_id', 'study_id', 'dicom_id'], keep=False)
merged_df = merged_df.drop_duplicates(subset=['subject_id', 'study_id', 'dicom_id'], keep=False)

merged_df = pd.concat([merged_df, filtered_duplicates])

print(len(merged_df))

In [ ]:
merged_df['year_difference'] = merged_df['StudyDate'].dt.year - merged_df['anchor_year']

# Adjust anchor_age based on the difference between anchor_year and StudyDate year
merged_df['adjusted_age'] = merged_df['anchor_age'] + merged_df['year_difference']

In [ ]:
merged_df.columns

In [ ]:
analysis_df = merged_df[['dicom_id', 'study_id', 'subject_id', 'split','gender', 'anchor_age', 'adjusted_age', 'marital_status', 'race']]
analysis_df

In [ ]:
213348/377110

In [ ]:
merged_df = merged_df = pd.merge(mimic_cxr_df, patients, on='subject_id', how='inner')
analysis_df = merged_df[['dicom_id', 'study_id', 'subject_id', 'split','gender', 'anchor_age']]
analysis_df.dropna()

In [ ]:
357542/377110

In [ ]:
321196 / 377110